<a href="https://colab.research.google.com/github/shamoji101/TensorFlow2.0Gym/blob/add_make_inception_with_tf20/Make_Inception_Network_with_TF20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# はじめに

今回の記事では、TF2.0付属のKerasModuleを用いて、様々な内容の
この記事はColabratoryで直接動かすことができます。
動かしながら確認したい方は是非利用してみてください。


# 新しく統合されたKerasと3つのAPIについて

KerasはTF2.0からcontribレイヤーから正式なTFの仲間入りをしました。
代わりにTF1.xにあった共有レイヤー機能などは削除されています。（つまりKerasで書け。ということみたいです）  
Kerasには色々なAPIがありますが、Model作成に関係するのは以下の3つのAPIです。

- Sequencial API
- Functional API
- SubClass API

これを使えば基本的に全てのモデルを作れるといってもいいくらい自由度が高いAPIになっています。
これらをCIFAR10モデルを例にチュートリアルしていこうと思います。

# 環境設定



## Colabの環境設定

まず、学習が重いので、  
[ランタイム]→[ランタイムのタイプを変更]を探して頂いて、  
Noneとなっている部分をGPUに変更するようにお願いします。

その後、以下のコードを実行します。


In [0]:
!pip install --upgrade tensorflow==2.0.0
!pip install --upgrade tensorflow-gpu=2.0.0


## import
インポートします。



In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import numpy as np 

print("TensorFlow Version:",tf.__version__)


# CIFAR10を扱う

今回チュートリアルで用いるのは皆さんお馴染みの[CIFAR10データセット](https://www.cs.toronto.edu/~kriz/cifar.html)です。  
データは(32,32,3)でのテンソルで表現されています。ダウンロードは`tf.keras`から行うことができます。

## データセットをダウンロード


In [0]:
(Train_X,Train_Y),(Test_X,Test_Y) = keras.datasets.cifar10.load_data()
Train_X,Test_X = Train_X/255.0, Test_X/255.0

##データの表示

一度表示させてみましょう。

In [0]:
import random

X = random.choice(Train_X)
print(X.shape)
plt.figure(facecolor="white")
plt.imshow(X)
plt.colorbar()
plt.grid(False)
plt.title("CIFAR10 sample")
plt.show()

次にランダムに沢山表示させてみましょう。

In [0]:
labels = np.array([
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'])
plt.figure(figsize=(10,30),facecolor="white")
index=random.randint(0,Train_Y.shape[0]-25)
for i in range(75):
    plt.subplot(15,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(Train_X[i+index])
    plt.xlabel(labels[Train_Y[i+index][0]])
plt.show()

分布を表示させてみましょう。

In [0]:
import collections
c=collections.Counter(Train_Y.reshape(-1))
c=sorted(c.items(),key=lambda x:x[0])
left = [label for label,count in c]
hight = [count for label,count in c]

plt.figure(facecolor="white")
plt.bar(left,hight,tick_label=left)
plt.show()

各ラベル、5000枚均一のようですね。


# チュートリアル１：KerasのSequentialAPIで問題を解く。


## SequentialAPI

SequentialAPIは、Kerasの中ではもっとも簡単なDNNModelの書き方で、
その分直列一直線にしかModelを組めません。

まずは複雑なモデルは組まず、単純なConv2Dモデルでやってみましょう。

SequentialAPIは基本的に以下のように記述します。



In [0]:
model=keras.Sequential()
model.add(keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same",input_shape=(32,32,3)))
model.add(keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same"))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="same"))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(filters=256,kernel_size=(3,3),padding="same"))
model.add(keras.layers.Conv2D(filters=256,kernel_size=(3,3),padding="same"))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation="relu"))
model.add(keras.layers.Dense(10,activation="sigmoid"))
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.summary()

このようにaddしていくことでDNNModelを構築することができます。

## 実際に学習させてみる

学習させてみましょう。やり方は簡単で、modelインスタンスの`.fit`を使用すればいいだけです。

In [0]:
history=model.fit(Train_X,Train_Y,validation_split=0.2,epochs=10,batch_size=128)

ちなみに、Colabで場合、自動的にGPUモードで学習してくれます。

評価は以下のように行うことができます。


In [0]:
model.evaluate(Test_X,Test_Y,batch_size=128,verbose=0)

左はloss、右はAccuracyを示しています。
## グラフの表示
グラフを表示させてみましょう。


In [0]:
# Accuracyを表示
plt.figure(figsize=(7,5),facecolor="white")
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# lossを表示
plt.figure(figsize=(7,5),facecolor="white")
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

お手本のような過学習が起きています。  
次こそはいいモデルを作るために、InceptionV3と呼ばれるモデルを作成してみましょう。

# チュートリアル2:Functional API
